# Creating Test and Train dataset

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv(r"C:\\Users\\ih2344\\Dropbox\\Raquel Senior - Jorge Guzman\\Imrul Stata\\bizowner.csv", encoding='latin-1')

C:\Users\ih2344\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cols_to_drop = ['idmaster', 'fullname', 'lname', 'fname', 'middleb', 'profession', 'districtb', 'u1', 'osu1type',
       'u2', 'hs', 'u1state', 'u1address', 'u1city', 'u1zip', 'addresscitystatezip', 'formatch', 
       'u1_lh2000', 'u1_lh2000name', 'u1_lh2010', 'u1_lh2010name',  
       'u1_uh2000','u1_uh2000name', 'u1_uh2010', 'u1_uh2010name',
       'degree1_code', 'degree2_code', 'x', 'y', 'statecode', 
       'local_degree', 'prob_entrepreneur', 'p', 'ceo', 'president', 'director', 'profession', 
       'districtb', 'districtnumb', 'dvtn', 'numberofwinners', 'termactualb', 'home']

In [4]:
reduced_df = data.drop(columns=cols_to_drop)
reduced_df = reduced_df.drop_duplicates()

In [5]:
ent_any = pd.DataFrame(reduced_df.groupby('idklarner').apply(lambda x: int(x['entrepreneur'].mean() > 0)))
ent_any['ent_any'] = ent_any[0]
reduced_df = reduced_df.merge(ent_any, on='idklarner', how='left')
# age variables
reduced_df['birth'] = pd.to_datetime(reduced_df['birth'], errors = 'coerce')
reduced_df['age_at_election'] = reduced_df['election'] - reduced_df['birth'].dt.year
age_first = pd.DataFrame(reduced_df.groupby('idklarner').apply(lambda x: x['age_at_election'].min()))
age_first['age_first'] = age_first[0]
reduced_df = reduced_df.merge(age_first, on='idklarner', how='left')
reduced_df['age_u1'] = reduced_df['u1year'] - reduced_df['birth'].dt.year
reduced_df['age_u1'] = reduced_df['age_u1'].apply(lambda x: x if x > 0 else None)
reduced_df['age_u2'] = reduced_df['u2year'] - reduced_df['birth'].dt.year
reduced_df['age_u2'] = reduced_df['age_u2'].apply(lambda x: x if x > 0 else None)
ran_for = pd.DataFrame(reduced_df.groupby('idklarner').apply(lambda x: 3 if x['chamberb'].nunique() > 1 else list(x['chamberb'])[0]))
ran_for['ran_for'] = ran_for[0]
reduced_df = reduced_df.merge(ran_for, on='idklarner', how='left')
ran_as = pd.DataFrame(reduced_df.groupby('idklarner').apply(lambda x: 5 if x['partyb'].nunique() > 1 else list(x['partyb'])[0]))
ran_as['ran_as'] = ran_as[0]
reduced_df = reduced_df.merge(ran_as, on='idklarner', how='left')
quality_df = pd.read_csv(r"C:\Users\ih2344\Dropbox\Raquel Senior - Jorge Guzman\Imrul Stata\state_2002_2014.csv")
reduced_df = reduced_df.merge(quality_df, left_on="state", right_on="datastate", how="left")
salary_df = pd.read_csv(r"C:\Users\ih2344\Dropbox\Raquel Senior - Jorge Guzman\Imrul Stata\salaries by legislators.csv")
salary_df['salary'] = salary_df['salary'].apply(lambda x: x.replace("$", "").replace(",","") if type(x) == str else x)
salary_df[' per_diem  '] = salary_df[' per_diem  '].apply(lambda x: x.replace("$", "").replace(",","") if type(x) == str else x)
salary_df['salary'] = salary_df['salary'].astype("float")
salary_df = salary_df[salary_df['salary'] > 0]
x = pd.DataFrame(salary_df['salary'])
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
salary_df['salary_norm'] = pd.DataFrame(x_scaled)
reduced_df = reduced_df.merge(salary_df, on="state", how="left")

In [6]:
numeric_variables = reduced_df.groupby("idklarner").agg({"ent_any": np.mean,
                                     "age_at_election": np.mean,
                                     "age_first": np.mean,
                                     "age_u1": np.mean,
                                     "age_u2": np.mean,
                                     "recpi": np.mean,
                                     "salary_norm": np.mean})

categorical_variables = reduced_df.groupby("idklarner").agg({"ent_any": "first",
                                     "ran_as": "first",
                                     "ran_for": "first",
                                     "gender": "first",
                                     "degree1": "first",
                                     "degree2": "first"})

In [7]:
categorical_variables['degree2'] = categorical_variables['degree2'].fillna("0")
categorical_variables['degree1'] = categorical_variables['degree1'].fillna("0")
party_dict = {1: "DEMOCRAT", 
              2: "NON-MAJOR PARTY", 
              3: "NON-PARTISAN ELECTION", 
              4: "REPUBLICAN", 
              5: "SWITCHED_PARTY"}
chamber_dict = {1: "LOWER-HOUSE", 
                2: "UPPER-HOUSE", 
                3: "BOTH-HOUSE"}
gender_dict = {1: "FEMALE", 
          2: "MALE"}
categorical_variables['ran_as'] = categorical_variables['ran_as'].apply(lambda x: party_dict[x])
categorical_variables['ran_for'] = categorical_variables['ran_for'].apply(lambda x: chamber_dict[x])
categorical_variables['gender'] = categorical_variables['gender'].apply(lambda x: gender_dict[x])

In [8]:
df = numeric_variables.merge(categorical_variables, left_index=True, right_index=True)

In [9]:
df = df[['age_at_election',
 'age_first',
 'recpi',
 'salary_norm',
 'ran_as',
 'ran_for',
 'gender',
 'degree1',
 'degree2','ent_any_x']]

In [10]:
degree_first = pd.get_dummies(df['degree1'],prefix="degree_1",drop_first=True)
degree_second = pd.get_dummies(df['degree2'],prefix="degree_2",drop_first=True)
gender = pd.get_dummies(df['gender'],drop_first=True)
ran_as = pd.get_dummies(df['ran_as'],drop_first=True)
ran_for = pd.get_dummies(df['ran_for'],drop_first=True)
df.drop(['degree1', 'degree2', 'gender', 'ran_as', 'ran_for'], axis=1, inplace=True)

In [11]:
df = pd.concat([df, degree_first, degree_second, gender, ran_as, ran_for],axis=1)

In [12]:
mean_salary = df['salary_norm'].mean()
mean_age_at_election = df['age_at_election'].mean()
mean_age_first = df['age_first'].mean()
mean_recpi = df['recpi'].mean()
df['salary_norm'] = df['salary_norm'].fillna(mean_salary)
df['age_at_election'] = df['age_at_election'].fillna(mean_age_at_election)
df['age_first'] = df['age_first'].fillna(mean_age_first)
df['recpi'] = df['recpi'].fillna(mean_recpi)

In [13]:
train, test = train_test_split(df, test_size=0.3, random_state=101)

In [14]:
train.shape

(11655, 257)

In [15]:
test.shape

(4995, 257)

In [16]:
train.to_csv("train.csv")
test.to_csv("test.csv")